# Timer

In [ ]:
from time import sleep
from datetime import datetime as dt

start_time = "25.05.2023, 21:50"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

ttype = 'sell'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '4h'
higher_timeframe = '1d'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'vol_window': [48], # [6, 12, 24, 48, 72, 96],
                            'vol_q_high': [0.7], # np.arange(0.1, 1, 0.05), 
                            'vol_q_low': [0.1], # np.arange(0.1, 1, 0.05),
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.5], # np.arange(0.5, 1, 0.05),
                            'second_candle': [0.85] # np.arange(0.5, 1, 0.05)
                            },
                'Trend': {
                            'timeperiod': [24], # np.arange(2, 50, 2),
                            'low_bound': [0]
                            }
             }


opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1d, trade type is sell
Number of combinations is 24


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [01:56<00:00,  4.85s/it]


# Check local statistics

In [3]:
# vol_q
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
216,Pattern_Trend,48,0.70,0.10,1,5,0.5,0.85,16,0,53,1.598863,-24.827500,31.739713,-1315.857500,1.3436,1.4129,1.5187,1.6161,1.5792,1.6786,1.7901,1.7910,1.8129,1.7338,1.6959,1.7211,1.7201,1.7138,1.6932,1.7142,1.7255,1.7150,1.7115,1.6810,1.6922,1.6394,0.8297,0.8432,-0.63,-1.40,-2.23,-3.17,-4.16,-5.12,-6.40,-7.66,-9.04,-10.54,-12.10,-14.16,-16.84,-20.42,-25.61,-34.83,-42.97,-50.57,-50.77,-50.47,-50.87,-54.83,-57.98,-63.09
217,Pattern_Trend,48,0.70,0.15,1,5,0.5,0.85,16,0,66,1.377025,-22.557500,24.883650,-1488.795000,1.2074,1.2620,1.3528,1.4640,1.4180,1.4863,1.5692,1.5593,1.5711,1.4888,1.4470,1.4664,1.4542,1.4601,1.4187,1.4338,1.4421,1.4249,1.4179,1.3976,1.4012,1.3416,0.7771,0.7871,-0.64,-1.39,-2.18,-3.10,-4.05,-4.96,-6.16,-7.33,-8.63,-10.03,-11.49,-13.31,-15.68,-18.81,-23.25,-31.03,-37.89,-45.46,-45.84,-45.81,-46.20,-49.67,-52.19,-56.28
198,Pattern_Trend,48,0.65,0.10,1,5,0.5,0.85,16,0,60,1.216150,-22.281250,12.969000,-1336.875000,1.3172,1.3981,1.5266,1.6203,1.5590,1.4244,1.5192,1.5249,1.5534,1.0802,1.0720,1.0907,1.0978,1.0955,1.0977,1.1107,1.1176,1.1171,1.1169,1.1092,1.1225,1.1033,0.7002,0.7131,-0.63,-1.35,-2.12,-2.99,-3.91,-4.78,-5.95,-7.14,-8.43,-9.77,-11.18,-13.04,-15.45,-18.65,-23.22,-31.27,-38.35,-44.87,-45.15,-45.01,-45.39,-48.87,-51.48,-55.75
218,Pattern_Trend,48,0.70,0.20,1,5,0.5,0.85,16,0,71,1.501896,-21.450833,35.634604,-1523.009167,1.2890,1.3667,1.4355,1.5535,1.5197,1.6446,1.7056,1.6912,1.6997,1.6195,1.5959,1.6209,1.6107,1.5959,1.5592,1.5726,1.5898,1.5710,1.5454,1.5250,1.5277,1.4635,0.8678,0.8751,-0.61,-1.33,-2.08,-2.95,-3.89,-4.77,-5.92,-7.04,-8.29,-9.64,-11.03,-12.81,-15.09,-18.04,-22.24,-29.53,-35.98,-42.97,-43.41,-43.44,-43.83,-47.13,-49.49,-53.31
199,Pattern_Trend,48,0.65,0.15,1,5,0.5,0.85,16,0,73,1.117362,-20.660417,8.567462,-1508.210417,1.1989,1.2618,1.3710,1.4792,1.4154,1.3106,1.3854,1.3818,1.4009,1.0186,1.0048,1.0199,1.0203,1.0251,1.0148,1.0264,1.0322,1.0267,1.0242,1.0176,1.0252,0.9972,0.6742,0.6845,-0.64,-1.35,-2.10,-2.95,-3.85,-4.69,-5.80,-6.93,-8.16,-9.45,-10.78,-12.47,-14.65,-17.51,-21.52,-28.49,-34.62,-41.17,-41.59,-41.67,-42.06,-45.18,-47.34,-50.88
200,Pattern_Trend,48,0.65,0.20,1,5,0.5,0.85,16,0,78,1.216475,-19.802083,16.885050,-1544.562500,1.2728,1.3583,1.4475,1.5616,1.5086,1.4434,1.5032,1.4961,1.5126,1.1111,1.1076,1.1266,1.1285,1.1232,1.1176,1.1286,1.1417,1.1368,1.1252,1.1188,1.1257,1.0945,0.7485,0.7569,-0.60,-1.29,-2.02,-2.83,-3.71,-4.54,-5.62,-6.70,-7.89,-9.14,-10.43,-12.08,-14.19,-16.91,-20.72,-27.31,-33.11,-39.24,-39.71,-39.84,-40.24,-43.23,-45.28,-48.62
219,Pattern_Trend,48,0.70,0.25,1,5,0.5,0.85,16,0,83,1.251225,-18.757083,20.851675,-1556.837917,1.1543,1.2171,1.2611,1.3583,1.3238,1.4232,1.4280,1.4234,1.4240,1.3582,1.3128,1.2898,1.3021,1.2904,1.2380,1.2538,1.2735,1.2516,1.2366,1.2305,1.2307,1.1650,0.7916,0.7916,-0.59,-1.27,-2.00,-2.84,-3.71,-4.52,-5.52,-6.50,-7.58,-8.73,-9.91,-11.37,-13.31,-15.81,-19.33,-25.46,-30.90,-37.46,-37.81,-37.81,-38.04,-40.89,-42.81,-46.00
180,Pattern_Trend,48,0.60,0.10,1,5,0.5,0.85,16,0,70,0.769021,-18.131250,-16.168542,-1269.187500,1.2356,1.3922,1.4087,1.4338,0.5

In [4]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank')
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
142,Pattern_Trend,48,0.45,0.90,1,5,0.5,0.85,16,0,238,0.819679,-9.816667,-42.916358,-2336.366667,1.0673,1.0031,0.9921,0.9664,0.7268,0.7755,0.7973,0.8242,0.8323,0.7721,0.7768,0.7805,0.7836,0.7726,0.7776,0.7828,0.7986,0.7788,0.7946,0.8095,0.8082,0.7986,0.7251,0.7279,-0.51,-1.03,-1.57,-2.14,-2.62,-3.19,-3.87,-4.53,-5.25,-5.92,-6.55,-7.32,-8.30,-9.45,-10.92,-13.23,-15.27,-17.18,-17.66,-18.09,-18.55,-19.87,-20.71,-21.87
196,Pattern_Trend,48,0.60,0.90,1,5,0.5,0.85,16,0,234,0.813200,-9.971250,-43.711200,-2333.272500,1.0716,1.0050,0.9895,0.9653,0.7312,0.7682,0.7867,0.8139,0.8215,0.7620,0.7673,0.7714,0.7747,0.7638,0.7691,0.7745,0.7907,0.7709,0.7871,0.8023,0.8010,0.7912,0.7175,0.7204,-0.53,-1.05,-1.61,-2.18,-2.68,-3.26,-3.95,-4.62,-5.35,-6.03,-6.67,-7.44,-8.44,-9.60,-11.08,-13.43,-15.51,-17.45,-17.94,-18.36,-18.82,-20.14,-21.00,-22.17
107,Pattern_Trend,48,0.35,0.95,1,5,0.5,0.85,16,0,242,0.807075,-9.641250,-46.687850,-2333.182500,1.0411,0.9877,0.9760,0.9506,0.7167,0.7659,0.7856,0.8121,0.8205,0.7606,0.7650,0.7688,0.7718,0.7611,0.7663,0.7714,0.7872,0.7667,0.7821,0.7967,0.7957,0.7863,0.7155,0.7184,-0.50,-1.02,-1.55,-2.10,-2.58,-3.14,-3.81,-4.46,-5.17,-5.83,-6.44,-7.19,-8.16,-9.30,-10.74,-13.01,-15.02,-16.87,-17.34,-17.75,-18.20,-19.48,-20.30,-21.43
70,Pattern_Trend,48,0.25,0.90,1,5,0.5,0.85,16,0,242,0.807075,-9.641250,-46.687850,-2333.182500,1.0411,0.9877,0.9760,0.9506,0.7167,0.7659,0.7856,0.8121,0.8205,0.7606,0.7650,0.7688,0.7718,0.7611,0.7663,0.7714,0.7872,0.7667,0.7821,0.7967,0.7957,0.7863,0.7155,0.7184,-0.50,-1.02,-1.55,-2.10,-2.58,-3.14,-3.81,-4.46,-5.17,-5.83,-6.44,-7.19,-8.16,-9.30,-10.74,-13.01,-15.02,-16.87,-17.34,-17.75,-18.20,-19.48,-20.30,-21.43
71,Pattern_Trend,48,0.25,0.95,1,5,0.5,0.85,16,0,242,0.807075,-9.641250,-46.687850,-2333.182500,1.0411,0.9877,0.9760,0.9506,0.7167,0.7659,0.7856,0.8121,0.8205,0.7606,0.7650,0.7688,0.7718,0.7611,0.7663,0.7714,0.7872,0.7667,0.7821,0.7967,0.7957,0.7863,0.7155,0.7184,-0.50,-1.02,-1.55,-2.10,-2.58,-3.14,-3.81,-4.46,-5.17,-5.83,-6.44,-7.19,-8.16,-9.30,-10.74,-13.01,-15.02,-16.87,-17.34,-17.75,-18.20,-19.48,-20.30,-21.43
35,Pattern_Trend,48,0.15,0.95,1,5,0.5,0.85,16,0,242,0.807075,-9.641250,-46.687850,-2333.182500,1.0411,0.9877,0.9760,0.9506,0.7167,0.7659,0.7856,0.8121,0.8205,0.7606,0.7650,0.7688,0.7718,0.7611,0.7663,0.7714,0.7872,0.7667,0.7821,0.7967,0.7957,0.7863,0.7155,0.7184,-0.50,-1.02,-1.55,-2.10,-2.58,-3.14,-3.81,-4.46,-5.17,-5.83,-6.44,-7.19,-8.16,-9.30,-10.74,-13.01,-15.02,-16.87,-17.34,-17.75,-18.20,-19.48,-20.30,-21.43
53,Pattern_Trend,48,0.20,0.95,1,5,0.5,0.85,16,0,242,0.807075,-9.641250,-46.687850,-2333.182500,1.0411,0.9877,0.9760,0.9506,0.7167,0.7659,0.7856,0.8121,0.8205,0.7606,0.7650,0.7688,0.7718,0.7611,0.7663,0.7714,0.7872,0.7667,0.7821,0.7967,0.7957,0.7863,0.7155,0.7184,-0.50,-1.02,-1.55,-2.10,-2.58,-3.14,-3.81,-4.46,-5.17,-5.83,-6.44,-7.19,-8.16,-9.30,-10.74,-13.01,-15.02,-16.87,-17.34,-17.75,-18.20,-19.48,-20.30,-21.43
89,Pattern_Trend,48,0.30,0.95,1,5,0.5,0.85,16,0,242,0.807075,-9.641250,-46.687850,-2333.182500,1.0411,0.9877,0.9760,0.9506,0.7167,0.7659,0.7856,0.8121,0

In [4]:
# window
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_Trend,24,0.7,0.1,1,5,0.5,0.85,16,0,37,1.490892,-32.478750,18.162992,-1201.713750,1.4143,1.5837,1.5300,1.5734,1.5599,1.6582,1.7591,1.6321,1.6158,1.5783,1.5832,1.6692,1.6338,1.6120,1.5379,1.5515,1.5552,1.5219,1.5035,1.4854,1.5284,1.4688,0.6043,0.6215,-0.89,-1.93,-3.03,-4.24,-5.44,-6.81,-8.22,-9.69,-11.30,-13.19,-15.24,-18.07,-21.63,-26.46,-33.45,-45.91,-56.89,-67.68,-67.72,-67.08,-66.59,-71.67,-74.62,-81.74
1,Pattern_Trend,12,0.7,0.1,1,5,0.5,0.85,16,0,40,0.434929,-27.227083,-22.602833,-1089.083333,1.1902,1.0769,0.2351,0.2700,0.2960,0.3199,0.3369,0.3443,0.3467,0.3409,0.3663,0.3856,0.3869,0.3879,0.3829,0.3839,0.3877,0.4031,0.4029,0.4053,0.4296,0.4275,0.4595,0.4723,-0.77,-1.67,-2.05,-2.89,-3.70,-4.70,-5.83,-7.06,-8.42,-10.10,-11.76,-14.03,-17.08,-21.42,-27.72,-39.24,-49.25,-57.75,-57.63,-57.00,-56.53,-61.16,-64.71,-70.98
3,Pattern_Trend,48,0.7,0.1,1,5,0.5,0.85,16,0,52,1.539304,-25.022917,28.043817,-1301.191667,1.3360,1.3841,1.4654,1.5586,1.4995,1.5948,1.7092,1.7147,1.7344,1.6599,1.6259,1.6533,1.6532,1.6489,1.6309,1.6521,1.6635,1.6538,1.6506,1.6214,1.6339,1.5831,0.8012,0.8149,-0.64,-1.41,-2.23,-3.17,-4.16,-5.10,-6.38,-7.63,-9.01,-10.51,-12.08,-14.16,-16.88,-20.53,-25.82,-35.23,-43.53,-51.30,-51.48,-51.13,-51.54,-55.56,-57.98,-63.09
0,Pattern_Trend,6,0.7,0.1,1,5,0.5,0.85,16,0,16,1.526650,-8.148750,8.426400,-130.380000,1.0368,1.0164,1.0380,1.5088,1.3901,1.4774,1.3609,1.5351,1.6656,1.5223,1.5477,1.5828,1.5876,1.6075,1.6102,1.5697,1.5697,1.5920,1.6879,1.7421,1.7471,1.7090,1.7643,1.7706,-0.43,-0.87,-1.32,-1.89,-2.48,-3.17,-3.85,-4.63,-5.50,-6.38,-6.10,-6.87,-7.73,-8.68,-9.63,-10.55,-11.42,-12.25,-13.07,-13.94,-14.90,-15.85,-16.60,-17.46
5,Pattern_Trend,96,0.7,0.1,1,5,0.5,0.85,16,0,49,0.603004,-7.122917,-19.452796,-349.022917,1.1797,1.2157,1.3392,1.3677,0.4378,0.4578,0.4869,0.5130,0.5338,0.4385,0.4413,0.4609,0.4639,0.4626,0.4719,0.4800,0.4839,0.4909,0.4923,0.4916,0.5031,0.5000,0.3754,0.3842,-0.59,-1.14,-1.73,-2.38,-2.57,-2.91,-3.57,-4.28,-4.99,-5.66,-6.21,-6.86,-7.59,-8.23,-8.95,-9.60,-10.21,-10.70,-11.28,-11.81,-12.10,-12.62,-12.25,-12.72
4,Pattern_Trend,72,0.7,0.1,1,5,0.5,0.85,16,0,61,0.616500,-6.940000,-23.393500,-423.340000,0.9842,1.0582,1.1607,1.2059,0.4564,0.4701,0.5082,0.5363,0.5525,0.4800,0.4920,0.5072,0.5164,0.5288,0.5391,0.5489,0.5529,0.5583,0.5592,0.5590,0.5702,0.5661,0.4383,0.4471,-0.54,-1.04,-1.59,-2.19,-2.44,-2.80,-3.42,-4.11,-4.83,-5.52,-6.10,-6.65,-7.38,-8.06,-8.77,-9.45,-10.06,-10.39,-10.95,-11.47,-11.80,-12.31,-12.12,-12.57


In [2]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
23,Pattern_Trend,48,0.7,0.1,1,5,0.5,0.85,48,0,24,1.460763,-44.542917,11.058300,-1069.030000,1.2741,1.3921,1.2974,1.4183,1.4463,1.5472,1.5815,1.5576,1.6300,1.6304,1.5041,1.5439,1.5439,1.4453,1.5659,1.6613,1.6394,1.6470,1.6347,1.6600,1.7049,1.7015,0.5114,0.5201,-0.75,-1.60,-2.56,-3.58,-4.70,-5.73,-7.41,-9.07,-11.04,-13.44,-16.23,-20.07,-25.41,-32.85,-44.01,-64.55,-82.50,-97.58,-97.26,-95.70,-94.33,-102.28,-111.13,-125.25
21,Pattern_Trend,48,0.7,0.1,1,5,0.5,0.85,44,0,25,1.502292,-44.196250,12.557292,-1104.906250,1.2346,1.3445,1.2755,1.3905,1.4239,1.6045,1.6729,1.6464,1.7140,1.7143,1.5871,1.6240,1.6240,1.5246,1.6138,1.7042,1.6829,1.6945,1.6859,1.7253,1.7687,1.7042,0.5431,0.5516,-0.91,-1.96,-3.08,-4.33,-5.60,-6.94,-8.87,-10.67,-12.69,-15.13,-17.80,-21.42,-26.46,-33.49,-44.01,-63.46,-80.52,-94.85,-94.58,-93.18,-92.02,-99.59,-107.94,-121.21
20,Pattern_Trend,48,0.7,0.1,1,5,0.5,0.85,42,0,27,1.486912,-44.139583,13.146637,-1191.768750,1.2123,1.3234,1.2601,1.3784,1.4118,1.5763,1.6438,1.6192,1.6847,1.6959,1.5740,1.6099,1.6099,1.5142,1.6017,1.6899,1.6692,1.6807,1.6724,1.7109,1.7534,1.6909,0.5472,0.5557,-0.91,-1.94,-3.04,-4.26,-5.52,-6.84,-8.72,-10.50,-12.50,-14.89,-17.50,-21.42,-26.46,-33.49,-44.01,-63.46,-80.52,-94.85,-94.58,-93.18,-92.02,-99.59,-107.94,-121.21
19,Pattern_Trend,48,0.7,0.1,1,5,0.5,0.85,40,0,28,1.744296,-43.200833,20.840283,-1209.623333,1.5609,1.7873,1.6491,1.7388,1.7433,1.8802,1.9348,1.8995,1.9646,1.9758,1.8270,1.8611,1.8611,1.7470,1.8232,1.9111,1.8878,1.8978,1.8873,1.9256,1.9671,1.8962,0.6140,0.6225,-0.90,-1.96,-3.06,-4.29,-5.55,-6.88,-8.76,-10.54,-12.54,-14.92,-17.53,-21.39,-26.32,-33.16,-43.31,-61.97,-78.35,-92.14,-92.01,-90.84,-89.92,-97.30,-105.29,-117.89
22,Pattern_Trend,48,0.7,0.1,1,5,0.5,0.85,46,0,26,1.477308,-41.351250,12.410017,-1075.132500,1.2515,1.2926,1.2353,1.3479,1.3832,1.5598,1.6275,1.6037,1.6688,1.6691,1.5504,1.5864,1.5864,1.4928,1.6017,1.6895,1.6689,1.6803,1.6783,1.7167,1.7589,1.6964,0.5504,0.5589,-0.78,-1.67,-2.66,-3.76,-4.90,-6.00,-7.70,-9.36,-11.22,-13.48,-15.99,-19.39,-24.17,-30.89,-40.98,-59.54,-75.80,-89.50,-89.31,-88.03,-86.96,-94.23,-101.91,-114.20
18,Pattern_Trend,48,0.7,0.1,1,5,0.5,0.85,38,0,30,1.692583,-39.931667,20.777500,-1197.950000,1.5288,1.8207,1.7103,1.8020,1.7819,1.9094,1.9604,1.8514,1.8946,1.9048,1.7744,1.7231,1.7231,1.6312,1.7039,1.7821,1.7861,1.7955,1.7871,1.8248,1.8622,1.8024,0.6268,0.6350,-0.86,-1.86,-2.92,-4.11,-5.35,-6.63,-8.42,-10.10,-11.99,-14.22,-16.62,-20.12,-24.62,-30.87,-40.13,-57.19,-72.18,-84.80,-84.75,-83.73,-82.94,-89.70,-96.63,-107.62
17,Pattern_Trend,48,0.7,0.1,1,5,0.5,0.85,36,0,32,1.792100,-39.600833,25.347200,-1267.226667,1.4741,1.9362,1.8865,1.9632,1.9289,2.0419,2.0864,1.9713,2.0124,2.0223,1.8846,1.8288,1.8288,1.7319,1.7991,1.8758,1.8786,1.8874,1.8783,1.9152,1.9516,1.8897,0.6646,0.6728,-0.84,-1.83,-2.89,-4.09,-5.31,-6.58,-8.32,-9.96,-11.80,-13.98,-16.32,-19.70,-24.07,-30.12,-39.06,-55.48,-69.90,-84.80,-84.75,-83.73,-82.94,-89.70,-96.63,-107.62
16,Pattern_Trend,48,0.7,0.1,1,5,0.5,0.85,34,0,34,1.924238,-37.652500,31.424075,-1280.185000,1.5132,1.9861,1.9624,2.03

# Research parameter influence

In [8]:
param = 'Pattern_second_candle'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_second_candle,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,0.5,0.854897,-0.304701,-168.318122,-286.847375,68130
1,0.6,0.850164,-0.288188,-150.008786,-258.243681,60370
2,0.7,0.849519,-0.281743,-133.779457,-224.315035,52007
3,0.8,0.839842,-0.299785,-109.302578,-205.237062,41617
4,0.9,0.871600,-0.322160,-72.804460,-170.303528,30776


# Plot result in file

In [11]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [7]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '4h'
higher_timeframe = '1d'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {
                            'vol_window': [48],
                            'vol_q_high': [0.7],
                            'vol_q_low': [0.1],
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.5],
                            'second_candle': [0.85],
                            },
                'Trend': {
                            'timeperiod': [20],
                            'low_bound': [0]
                            }
             }
 
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)